# Get Committee Data from ProPublica's Campaign Finance API #  
This notebook queries ProPublica's [Campaign Finance API](https://propublica.github.io/campaign-finance-api-docs/) for available committee data.  
The API includes committees that file with the Federal Election Commission (FEC).

For more info on ProPublica check out https://propublica.org.

In [ ]:
import json
import pandas as pd
import requests
import time
import pickle
import pprint as pp

In [ ]:
def get_request(committee):
    headers = {
    'X-API-Key': '{}'.format(API_key),
    }

    r = requests.get('https://api.propublica.org/campaign-finance/v1/2012/committees/{}'.format(committee) + '.json', 
                     headers=headers)
    time.sleep(1)
    
    return json.loads(r.text)


def gather_committees(committees):
    return {committee: get_request(committee) for committee in committees}

### Load in dictionary of committees from the Free the Files API that file with the FEC ###

In [ ]:
fec = pickle.load(open('../pickle_jar/fec_committees.p','rb'))

### Query Campaign Finance API for data on these committees ###

In [ ]:
committee_data = gather_committees(list(fec.values()))
pickle.dump(committee_data,open('../pickle_jar/cf_committee_data.p','wb'))

### Build dictionary of committee data and dump to csv ###

In [ ]:
i = 0
committee_data_dict = {}
for key in committee_data.keys():
    committee_data_dict[i] = committee_data[key]['results'][0]
    i += 1

In [ ]:
# Remove 'other_cycles' field, which lists other election years a committee filed with the FEC
# I'm only interested in 2012, which is the year the Free the Files data is from 

for v in committee_data_dict.values():
    try:
        del v['other_cycles']
    except:
        pass

In [ ]:
pd.DataFrame.from_dict(committee_data_dict,orient='index').to_csv('../data/cf_committee_data.csv')